# Imported all the required libraries

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import nltk
import re
import scikitplot as skplt
import string
import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout,BatchNormalization, Activation,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB as mnb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

# removed stopwords,punctuation and appended the dataset

In [0]:
def stopwords_remove(text):
    stop = stopwords.words('english')
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def append(f)
    sent=[]
    cat=[]
    final=''
    i=0
    for x in f:
      if len(x)>1 and 'meta' == x.split()[0] and ('negative' == x.split()[-1]):
        if(i!=0):
          sent.append(final)
          final=''

        cat.append('negative')
      elif len(x)>1 and 'meta' == x.split()[0] and ( 'positive' == x.split()[-1]):
        if(i!=0):
          sent.append(final)
          final=''
        cat.append('positive')
      elif len(x)>1 and 'meta' == x.split()[0]and ( 'neutral' == x.split()[-1]):
        if(i!=0):
          sent.append(final)
          final=''
        cat.append('neutral')

      elif len(x)>1:
        final=final+' ' + x.split()[0]
      i=i+1 
    sent.append(final)
    
f = open("train.txt", "r")
append(f)


# did the same procedure for test data

In [0]:
df=pd.DataFrame(columns=['sent','cat'])
df['sent']=sent
df['cat']=cat

f = open("test.txt", "r")
append(f)

In [0]:
tdf=pd.DataFrame(columns=['sent','cat'])
tdf['sent']=sent
tdf['cat']=cat

In [11]:
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

df['sent']=df['sent'].str.lower()
tdf['sent']=tdf['sent'].str.lower()
df['sent'] = df['sent'].apply(remove_punctuation)
tdf['sent']=tdf['sent'].str.lower()
tdf['sent'] = tdf['sent'].apply(remove_punctuation)
df['sent'] = df['sent'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
tdf['sent'] = tdf['sent'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [0]:
X_train, X_test11, y_train, y_test11 = train_test_split(df['sent'], df['cat'], test_size=1)
X_test, X_cv, y_test, y_cv = train_test_split(tdf['sent'], tdf['cat'], test_size=0.1)



# dimensions for my neural language model

In [ ]:
# list of most frequent words
MAX_NB_WORDS = 12000
# Sequence length
MAX_SEQUENCE_LENGTH = 500
# Embedding dimensions
EMBEDDING_DIM = 1000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['sent'].values)
word_index = tokenizer.word_index


In [ ]:
X = tokenizer.texts_to_sequences(df['sent'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
X1 = tokenizer.texts_to_sequences(tdf['sent'].values)
X1 = pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH)


# encoded it into a one hot encoder

In [0]:

enc = OneHotEncoder(handle_unknown='ignore')
arry=np.array(df['cat']).reshape(-1,1)
enc.fit(arry)
Y=enc.transform(arry)


In [0]:
enc = OneHotEncoder(handle_unknown='ignore')
arry=np.array(tdf['cat']).reshape(-1,1)
enc.fit(arry)
Y1=enc.transform(arry)

In [ ]:
X_train, X_test111, Y_train, Y_test11 = train_test_split(X,Y, test_size =1)
X_test, X_cv, y_test, y_cv = train_test_split(X1, Y1, test_size=0.10)


In [70]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(Dropout(0.36))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

epochs = 5
batch_size = 700

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.08)#callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 300, 800)          8000000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 300, 800)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               360400    
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 303       
Total params: 8,360,703
Trainable params: 8,360,703
Non-trainable params: 0
_________________________________________________________________
Train on 13919 samples, validate on 1211 samples
Epoch 1/5
13919/13919 [==============================] - 32s 2ms/step - loss: 1.0332 - acc: 0.4454 - val_loss: 0.8857 - val_acc: 0.5632
Epoch 2/5
13919/13919 [==============================] - 

# Training Accuracy

In [71]:
accr = model.evaluate(X_train,Y_train)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

15130/15130 [==============================] - 127s 8ms/step
Test set
  Loss: 0.416
  Accuracy: 0.855


# Test Accuracy

In [72]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1682/1682 [==============================] - 15s 9ms/step
Test set
  Loss: 1.198
  Accuracy: 0.534


# Performance Metrics for evaluation

In [78]:
newy_p=enc.inverse_transform(y_test)
le = preprocessing.LabelEncoder()
le.fit(newy_p)
newy=model.predict_classes(X_test)
clas=le.inverse_transform(newy)
clas1=newy_p.flatten()
target_names = ['Positive', 'Neutral', 'Negative']
print(classification_report(clas,clas1, target_names=target_names))

              precision    recall  f1-score   support

    Positive       0.56      0.56      0.56       486
     Neutral       0.44      0.52      0.47       573
    Negative       0.63      0.53      0.58       623

    accuracy                           0.53      1682
   macro avg       0.54      0.54      0.54      1682
weighted avg       0.55      0.53      0.54      1682

